In [ ]:
import pandas as pd
import numpy as np

In [ ]:
game_df = pd.read_csv("raw_data/Twitch_game_data.csv")
global_df = pd.read_csv("raw_data/Twitch_global_data.csv")
streamers_df = pd.read_csv("raw_data/twitchdata-update.csv")

In [ ]:
def dateToDateTime(dataframe):
    df = dataframe.copy()
    df["day"] = 1
    df = df.set_index(pd.to_datetime(df[["year", "Month", "day"]]))
    df = df.drop(["year", "Month", "day"], axis=1)
    return df

## Global viewers chart

In [ ]:
Viewers = global_df[["year","Month","Avg_viewers"]]

In [ ]:
Viewers = dateToDateTime(Viewers)

In [ ]:
Viewers.to_csv("data/global_viewers.csv")

## Games views for ridgeplot

In [ ]:
game_df.drop(game_df[game_df["Game"] == "Just Chatting"].index, inplace=True)

In [ ]:
game_df.drop(columns=["Rank","Hours_watched","Hours_Streamed","Peak_viewers","Peak_channels","Streamers","Avg_channels","Avg_viewer_ratio"] ,inplace=True)

In [ ]:
game_df["day"] = 1
game_df["date"] = pd.to_datetime(game_df[["Year", "Month", "day"]])
game_df = game_df.drop(["Year", "Month", "day"], axis=1)

In [ ]:
top_games = game_df.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False)[:10].index.to_list()

In [ ]:
games = []

for game in top_games:
    games.append(game_df[game_df["Game"] == game])

In [ ]:
for g in games:
    g.rename(columns={"Avg_viewers" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)

In [ ]:
from functools import reduce

pd.concat(games)

top_games_avg_viewers = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [ ]:
top_games_avg_viewers.to_csv("data/top_games_viewers.csv")

for i,game in enumerate(games):
    game.to_csv("data/"+ games[i]["Game"].iloc[0] +".csv")